<div style="background-color:dodgerblue;height:100px;"><br>
<center><h1><font style="color:white">The Path to Self-Discovery: Build Structure, Get Meaning</font></h1></center>
</div>

# Looking Back at the Past

In order to move forward, we need to remember where we have been.

When we left off, we had a list of records based off of our sample data.

** As we said in the last lesson, all of this parsing is based off of assumptions.  Changes were made to the function below to account for some new conditions.**

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(open('./My Activity - Sample.html', 'r', encoding='iso8859-1').read(), 'html.parser')

In [ ]:
import re

location_pattern = '\?q=([^\s]+?),([^\s]+)'

def parse_record(element):
    ### need to properly parse out the search target info.  first line, split on the visited or serached for, take the text from rest
    section1 = element.find("div", "content-cell")
    
    if section1.find(text=re.compile('Visited|Searched')):
        section1_strings = list(section1.strings)
        activity_type = None
        target = None
        datestamp = None
        
        if len(section1_strings)==3:
            activity_type, target, datestamp = section1_strings
        elif len(section1_strings)==2:
            activity_type, target = section1_strings[0].split('\xa0')
            datestamp = section1_strings[-1]
        else:
            print('---Unknown----')
            section1_strings
            return None
        activity_type = activity_type.replace('\xa0', '').strip()

        section2 = element.select("div.content-cell.mdl-typography--caption")[0]
        section_strings = list(section2.strings)
        product = section_strings[1].replace('\u2003','').strip()
        lat = None
        lon = None

        if 'Locations:' in section_strings:
            location_url = section2.find("a",{"href":re.compile("https://google.com/maps\?q=")}).text
            match = re.search(location_pattern, location_url)
            if match:
                lat, lon = match.groups()

        data = {
            'Activity Type': activity_type,
            'Target': target,
            'Datestamp': datestamp,
            'Product': product,
            'Lat':lat,
            'Lon':lon
        }

        return data
    
    else:
        return None

my_records = []

for record in soup.find_all('div', 'outer-cell'):
    new_record = parse_record(record)
    if new_record:
        my_records.append(new_record)
    
for record in my_records:
    print(record)

## Exercise 1

We ran this function over a sample dataset, but now we need to do it over the whole shabang!  Using everything you learned from the last lesson (and cheating with the example above), parse the records out of the full data file.  This is where you can use your own, or the dummy dataset we provided!

In [ ]:
full_records = []

# YOUR WORK HERE


In [ ]:
new_soup = BeautifulSoup(open('./MyActivity.html', 'r', encoding='iso8859-1').read(), 'html.parser')

for record in new_soup.find_all('div', 'outer-cell'):
    new_record = parse_record(record)
    if new_record:
        full_records.append(new_record)

In [ ]:
print(len(full_records))
print(full_records[0:10])

Now we have a looot more records to deal with.  And they certainly aren't the easiest to read or navigate.  What we really want is a pretty, matrix structure, so that we can look at both the rows and columns.  A list of dictionaries is not quite that.  So what do we do?

In [ ]:
full_records.count(None)

# Introducing Pandas

Pandas is a Python library that makes it easy to load and manipulate 1-3 dimensional data.  These objects are called:
* 1-D: Series
* 2-D: DataFrame
* 3-D: Panel

We can generate these objects from:
* CSV
* Excel
* HTML
* List of dictionaries
* List of lists
* ..and more

You should read all about how to create DataFrames, but since we have already built our data, the DataFrame constructor knows how to take our list of dictionaries in and make it pretty!

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(full_records)

We can preview this DataFrame using the `head` function.

In [ ]:
df.head()

In [ ]:
df.head(2)

Or we can grab the end of our DataFrame with `tail`.

In [ ]:
df.tail(3)

# Starting Simple

Right off the bat, pandas gives us some quick ways to scope our DataFrame.  Let's explore some of these basic operations.

In [ ]:
len(df)

In [ ]:
df.size

In [ ]:
df.shape

In [ ]:
df.shape[0]*df.shape[1]

In [ ]:
len(df.columns)

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
df.count()

In [ ]:
df['Activity Type'].count()

# The Data Type Cleanse

We can list the data types of each column using the `dtypes` command.

In [ ]:
df.dtypes

In [ ]:
df.head()

We can also change a columns data type, or cast it as a new type, using the `astype` command.

In [ ]:
df['Lat'] = df['Lat'].astype(float)
df['Lon'] = df['Lon'].astype(float)
df.dtypes

We stil would like to get the Datestamp column in a more friendly format, though.  The way it is now could trip us up, because calculations will be performed as if the column is a string.

In [ ]:
df['Datestamp'].min()

In [ ]:
df.Datestamp.max()

Luckily, pandas `to_datetime` command easily parses our datestamps without us having to do any date formatting!

In [ ]:
df['Datestamp'] = pd.to_datetime(df['Datestamp'])

In [ ]:
df['Datestamp'].min()

In [ ]:
df['Datestamp'].max()

# Filtering Out the Muck

Pandas also allows us to easily filter our data!  We can use some really simple syntax for this.

In [ ]:
df.head()

In [ ]:
df[df['Target']=='download "my activity" google']

In [ ]:
df[df['Lat'].isnull()].head()

# Asking Questions of Our Data Using Pandas

Now, armed with this small set of tools, let's use pandas to answer some questions we would commonly have about our data!

## Exercise 2
How many rows of data are in the dataset?

## Exercise 3

How many search activities searched for 'google drive'?

## Exercise 4

What is the median timestamp in the dataset?

## Exercise 5

What are the average latitude and longitude?

## <i> Challenge Question</i>: Exercise 6 

What was the top search made on July 4, 2017?

# Save Your Status

So that we can use our data again, let's save off what we've done so far!

In [ ]:
df.to_csv('MyStructuredSearch.csv', index=False, encoding='utf-8')